<a href="https://colab.research.google.com/github/MuhammadAteeb225/Complete-Deep-Learning-Course/blob/main/Optimizing_CNN_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created By: [Muhammad Ateeb](https://github.com/MuhammadAteeb225)

# CNN Model Optimazation Using Keras Tuner
[Keras Tuner API Reference](https://keras.io/api/keras_tuner/)

In [3]:
!pip install keras-tuner

     |████████████████████████████████| 96 kB 4.4 MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-py3-none-any.whl size=9568 sha256=fecfbd20d12b51e223cbcb2452eb89f0b5c99000196d8f5637da06be4b04caf1
  Stored in directory: /root/.cache/pip/wheels/38/5c/e0/13003e68c17f403af40b92a24d20171b95fef13b0fdaba833c
Successfully built kt-legacy


In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from kerastuner.engine.hyperparameters import HyperParameters as hp 

In [35]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [36]:
train_images = train_images/255.0
test_images = test_images/255.0

In [37]:
train_images.shape

(60000, 28, 28)

In [38]:
train_images = train_images.reshape(len(train_images), 28, 28,1)
test_images = test_images.reshape(len(test_images), 28, 28,1)

In [39]:
def build_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.Conv2D(
      filters=hp.Int("conv_1_filter", min_value=64, max_value=128, step=16),
      kernel_size=hp.Choice(name='conv_1_kernal',values=[3,5]),
      activation='relu',
      input_shape=(28,28,1))
  )
  model.add(keras.layers.Conv2D(
      filters=hp.Int(name='conv_2_filter', min_value=32, max_value=64, step=16),
      kernel_size= hp.Choice(name='conv_2_kernal',values=[3,5]),
      activation='relu')
  )
  model.add(
      keras.layers.Flatten()
  )
  model.add(
      keras.layers.Dense(
          units=hp.Int(name='dense_1_units',min_value=32 , max_value=128 , step=16), 
          activation='relu'
        )
  )
  model.add(
      keras.layers.Dense(units=10, activation='softmax')
  )
  model.compile(
      optimizer = keras.optimizers.Adam(learning_rate=hp.Choice(name='learning_rate',values=[1e-2, 1e-3])),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy']
  )

  return model

In [42]:
tuner_search = RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=5,
    project_name='mnist fashion')

In [43]:
tuner_search.search(train_images,train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 24s]
val_accuracy: 0.9128333330154419

Best val_accuracy So Far: 0.9128333330154419
Total elapsed time: 00h 02m 54s
INFO:tensorflow:Oracle triggered exit


In [44]:
best_model = tuner_search.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        34608     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1769536   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,805,594
Trainable params: 1,805,594
Non-trainable params: 0
_________________________________________________________________


In [45]:
best_model.fit(train_images,train_labels,validation_split=0.1, initial_epoch=3,epochs=10)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1329 - accuracy: 0.9504 - val_loss: 0.2549 - val_accuracy: 0.9082
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0936 - accuracy: 0.9654 - val_loss: 0.2936 - val_accuracy: 0.9125
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0681 - accuracy: 0.9748 - val_loss: 0.3519 - val_accuracy: 0.9082
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0468 - accuracy: 0.9829 - val_loss: 0.3777 - val_accuracy: 0.9142
Epoch 8/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0372 - accuracy: 0.9864 - val_loss: 0.4418 - val_accuracy: 0.9115
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0282 - accuracy: 0.9896 - val_loss: 0.4681 - val_accuracy: 0.9093
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0243 - accuracy: 0.9916 - val_loss: 0.4366 - val_accuracy